In [9]:
import numpy as np
import tensorflow as tf
import cv2
import networkx as nx
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input


# Training Data
X = np.array([[0, 0, 1],
              [1, 1, 1],
              [1, 0, 1],
              [0, 1, 1],
              [0, 1, 0],
              [1, 0, 1],
              [1, 0, 0]], dtype=np.float32)
Y = np.array([[0], [1], [1], [1], [0], [1], [0]], dtype=np.float32)

# Create and train the model
model = Sequential()
model.add(Input(shape=(3,)))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X, Y, epochs=1000, verbose=0)

# Define a tf.function to reduce retracing
@tf.function(reduce_retracing=True)
def predict_model(X):
    return model(X)

# Get predictions and weights
prediction = predict_model(X)
weights, biases = model.layers[0].get_weights()

def draw_neural_network():
    layers = [3, 1]  # 3 input neurons, 1 output neuron
    G = nx.DiGraph()
    pos = {}
    y_offset = 1.5
    x_offset = 1.0
    node_counter = 0
    layer_x_positions = [i * y_offset for i in range(len(layers))]
    
    for layer_idx, num_neurons in enumerate(layers):
        x = layer_x_positions[layer_idx]
        for neuron_idx in range(num_neurons):
            y = neuron_idx * x_offset - (num_neurons - 1) * x_offset / 2
            pos[node_counter] = (x, y)
            G.add_node(node_counter)
            node_counter += 1
    
    edge_labels = {}
    for i in range(3):
        G.add_edge(i, 3, weight=weights[i, 0])
        edge_labels[(i, 3)] = f'{weights[i, 0]:.2f}'
    
    plt.figure(figsize=(6, 6))
    nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightblue', edge_color='gray')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    
    plt.title(f"Simple Neural Network Graph\nBias: {biases[0]:.2f}")
    plt.show()

draw_neural_network()


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.